# 如何从工具返回构件

:::info 先决条件
本指南假定您熟悉下列概念：

- [ToolMessage](/docs/concepts/messages/#toolmessage)
- [Tools](/docs/concepts/tools)
- [Function/tool calling](/docs/concepts/tool_calling)

:::

[Tools](/docs/concepts/tools/) 是可以 [被模型调用](/docs/concepts/tool_calling/) 的实用程序，其输出被设计为反馈给模型。然而，有时工具执行的构件，我们希望使其可被链或代理中的下游组件访问，但又不想将其暴露给模型本身。例如，如果一个工具返回一个自定义对象、一个数据帧或一个图像，我们可能希望将该输出的一些元数据传递给模型，而无需将实际输出传递给模型。同时，我们可能希望能够在其他地方访问这个完整的输出，例如在下游工具中。

Tool 和 [ToolMessage](https://python.langchain.com/api_reference/core/messages/langchain_core.messages.tool.ToolMessage.html) 接口使得区分工具输出中供模型使用的部分（即 ToolMessage.content）和供模型外部使用的部分（ToolMessage.artifact）成为可能。

:::info 需要 ``langchain-core >= 0.2.19``

此功能已添加到 ``langchain-core == 0.2.19`` 中。请确保您的包是最新的。

:::

## 定义工具

如果我们希望我们的工具区分消息内容和其他构件，我们需要在定义工具时指定 `response_format="content_and_artifact"` 并确保我们返回一个元组 (content, artifact)：

In [ ]:
%pip install -qU "langchain-core>=0.2.19"

In [2]:
import random
from typing import List, Tuple

from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def generate_random_ints(min: int, max: int, size: int) -> Tuple[str, List[int]]:
    """Generate size random ints in the range [min, max]."""
    array = [random.randint(min, max) for _ in range(size)]
    content = f"Successfully generated array of {size} random ints in [{min}, {max}]."
    return content, array

## 使用 ToolCall 调用工具

如果我们直接只使用工具参数来调用我们的工具，您会注意到我们只会收到 Tool 输出的内容部分：

In [3]:
generate_random_ints.invoke({"min": 0, "max": 9, "size": 10})

'Successfully generated array of 10 random ints in [0, 9].'

为了取回内容和 artifact，我们需要用 ToolCall 来调用我们的模型（它只是一个包含 "name"、"args"、"id" 和 "type" 键的字典），其中包含生成 ToolMessage 所需的附加信息，如工具调用 ID：

In [4]:
generate_random_ints.invoke(
    {
        "name": "generate_random_ints",
        "args": {"min": 0, "max": 9, "size": 10},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

ToolMessage(content='Successfully generated array of 10 random ints in [0, 9].', name='generate_random_ints', tool_call_id='123', artifact=[2, 8, 0, 6, 0, 0, 1, 5, 0, 0])

## 与模型协同使用

通过我们提供的 [工具调用模型](/docs/how_to/tool_calling/)，我们可以轻松地让模型调用我们的 `Tool` 并生成 `ToolMessage`：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>

In [5]:
# | echo: false
# | output: false

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

In [6]:
llm_with_tools = llm.bind_tools([generate_random_ints])

ai_msg = llm_with_tools.invoke("generate 6 positive ints less than 25")
ai_msg.tool_calls

[{'name': 'generate_random_ints',
  'args': {'min': 1, 'max': 24, 'size': 6},
  'id': 'toolu_01EtALY3Wz1DVYhv1TLvZGvE',
  'type': 'tool_call'}]

In [7]:
generate_random_ints.invoke(ai_msg.tool_calls[0])

ToolMessage(content='Successfully generated array of 6 random ints in [1, 24].', name='generate_random_ints', tool_call_id='toolu_01EtALY3Wz1DVYhv1TLvZGvE', artifact=[2, 20, 23, 8, 1, 15])

如果我们只传入 tool call args，我们将只收到内容：

In [8]:
generate_random_ints.invoke(ai_msg.tool_calls[0]["args"])

'Successfully generated array of 6 random ints in [1, 24].'

如果我们想声明性地创建一个链，我们可以这样做：

In [9]:
from operator import attrgetter

chain = llm_with_tools | attrgetter("tool_calls") | generate_random_ints.map()

chain.invoke("give me a random number between 1 and 5")

[ToolMessage(content='Successfully generated array of 1 random ints in [1, 5].', name='generate_random_ints', tool_call_id='toolu_01FwYhnkwDPJPbKdGq4ng6uD', artifact=[5])]

## 从 BaseTool 类创建

如果你想直接创建一个 BaseTool 对象，而不是使用 `@tool` 装饰器来装饰一个函数，你可以这样做：

In [10]:
from langchain_core.tools import BaseTool


class GenerateRandomFloats(BaseTool):
    name: str = "generate_random_floats"
    description: str = "Generate size random floats in the range [min, max]."
    response_format: str = "content_and_artifact"

    ndigits: int = 2

    def _run(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
        range_ = max - min
        array = [
            round(min + (range_ * random.random()), ndigits=self.ndigits)
            for _ in range(size)
        ]
        content = f"Generated {size} floats in [{min}, {max}], rounded to {self.ndigits} decimals."
        return content, array

    # Optionally define an equivalent async method

    # async def _arun(self, min: float, max: float, size: int) -> Tuple[str, List[float]]:
    #     ...

In [11]:
rand_gen = GenerateRandomFloats(ndigits=4)
rand_gen.invoke({"min": 0.1, "max": 3.3333, "size": 3})

'Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.'

In [12]:
rand_gen.invoke(
    {
        "name": "generate_random_floats",
        "args": {"min": 0.1, "max": 3.3333, "size": 3},
        "id": "123",
        "type": "tool_call",
    }
)

ToolMessage(content='Generated 3 floats in [0.1, 3.3333], rounded to 4 decimals.', name='generate_random_floats', tool_call_id='123', artifact=[1.5789, 2.464, 2.2719])